In [1]:
import datetime
import json
import pandas as pd
import numpy as np
import spacy
import re
import math
from unidecode import unidecode
from collections import Counter
from pymongo import MongoClient
nlp = spacy.load("es_core_news_lg")

C:\Users\anama\anaconda3\envs\topicmodelin\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
client = MongoClient('mongodb://localhost:27017')
database = client['Thesis']
collectionDB = database.get_collection("text")
collectionWords = database.get_collection("words")
collectionDep_1 = database.get_collection("text_dep1")

In [54]:
def preproccess_0(texto):
    minusculas = texto.lower()
    sin_tildes = unidecode(minusculas)

    # Step 1: Eliminar caracteres especiales.
    texto_sin_caracteres_especiales = re.sub(r"[^a-zA-Z0-9áéíóúüñÁÉÍÓÚÜÑ\s]", "", sin_tildes)
    # Step 2: Eliminar palabras con más de 20 caracteres o con menos de 2.
    palabras = texto_sin_caracteres_especiales.split()
    palabras_filtradas = []
    for palabra in palabras:
        if len(palabra) > 1 and len(palabra) <= 20:
            palabras_filtradas.append(palabra)
    texto_filtrado = " ".join(palabras_filtradas)
    
    # Step 3: Eliminar stopwords.
    #doc = nlp(texto_filtrado)
    #palabras_sin_stopwords = [token.text for token in doc if not token.is_stop]
    #texto_sin_stopwords = " ".join(palabras_sin_stopwords)   
    
    if len(texto_filtrado) > 1000000:
        sin_tildes = " "
        for i in range(math.ceil(len(text)/1000000)):
            a = math.ceil(len(text)/math.ceil(len(text)/1000000))
            text_a = texto_filtrado[i*a:(i+1)*a]
            doc = nlp(text_a)
            palabras_lematizadas = [token.lemma_ for token in doc]
            text_lemma = " ".join(palabras_lematizadas)
            text_unicode = unidecode(text_lemma)
            sin_tildes = sin_tildes + " " +  text_unicode
    else:
        # Step 3: Lematizar las palabras.
        doc = nlp(texto_filtrado)
        palabras_lematizadas = [token.lemma_ for token in doc]
        text_lemma = " ".join(palabras_lematizadas)
        sin_tildes = unidecode(text_lemma)     
    return sin_tildes

In [55]:
print("Start  -------------------------------")
print(datetime.datetime.now().time())
count = 0
for document in collectionDB.find():
    text = document["text"]
    number = document["file"]
    texto = preproccess_0(text)
    frequency = pd.Series(re.findall(r'\w+', texto)).value_counts().to_dict()
    res = dict()
    res['version'] = "0.0.1"
    res['file'] = number
    res['freqWords'] = frequency
    result_object = collectionWords.insert_one(res)
    
print("End -------------------------------")
print(datetime.datetime.now().time())

Start  -------------------------------
21:40:47.212055


C:\Users\anama\AppData\Local\Temp\ipykernel_47324\1816659237.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  frequency = pd.Series(re.findall(r'\w+', texto)).value_counts().to_dict()
C:\Users\anama\AppData\Local\Temp\ipykernel_47324\1816659237.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  frequency = pd.Series(re.findall(r'\w+', texto)).value_counts().to_dict()
C:\Users\anama\AppData\Local\Temp\ipykernel_47324\1816659237.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  frequency = pd.Series(re.findall(r'\w+', texto)).value_counts().to_dict()
C:\Users\anama\AppData\Local\Temp\ipykernel_47324\1816659237.py:8: FutureWarning: The de

End -------------------------------
10:30:27.285913


In [57]:
print("Start  -------------------------------")
print(datetime.datetime.now().time())
count = 0
for document in collectionDB.find():
    text = document["text"]
    number = document["file"]
    texto = preproccess_0(text)
    res = dict()
    res['version'] = "0.0.1"
    res['file'] = number
    res['text'] = texto
    result_object = collectionDep_1.insert_one(res)
    
print("End -------------------------------")
print(datetime.datetime.now().time())

Start  -------------------------------
15:45:14.054309
End -------------------------------
04:44:46.322930


In [ ]:
data.columns